# Deterministic Uncertainty Sets - Max of affine uncertainty, ellipsoidal set, mean robust set

In [1]:
import cvxpy as cp
import scipy as sc
from sklearn import datasets
import numpy as np
import numpy.random as npr
import numpy.testing as npt
import torch
import pandas as pd
import lropt
import matplotlib.pyplot as plt
RTOL = 1e-04
ATOL = 1e-04
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

We solve the problem
$$
\begin{array}{ll}
		\text{minimize} & \tau \\
		\text{subject to } & \max\{a^Tx - d^Tx, a^Tx - d^T(3u+3)\} \leq \tau,\\
\end{array}
$$
where $a$ and $d$ are constants, and $u$ is the uncertain parameter from an ellipsoidal uncertainty set. 

In [2]:
m = 5
# Generate data
data = np.random.normal(0,1,size = (100,m))
# formulate uncertainty set
n = 5
u = lropt.UncertainParameter(n,
                             uncertainty_set=lropt.Ellipsoidal(p=2, rho=1, b = -np.mean(data, axis = 0)))
# formulate cvxpy variables
x_r = cp.Variable(n)
t = cp.Variable()

# formulate constants
a = npr.uniform(1,4,n)
d = a + npr.uniform(2,5,n)

# formulate objective
objective = cp.Minimize(t)

# formulate constraints 
constraints = [cp.maximum(a@x_r - d@x_r, a@x_r - d@(3*u+3)) <= t]
constraints += [x_r >= 0]


# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x_r.value)

LRO objective value:  -29.61311182296922 
LRO x:  [ 6.82227780e+00 -5.31982565e-10 -4.06531654e-10 -1.07906326e-09
  5.14566555e-10]


We assert that it is equivalent to the direct reformulation

In [3]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
t = cp.Variable()

# formulate objective
objective = cp.Minimize(t)

# formulate constraints
constraints = [a@x_cvxpy -d@x_cvxpy <= t]
constraints += [a@x_cvxpy -3*d@np.ones(n) + np.mean(data, axis = 0)@(-3*d) + cp.norm(3*d,2) <= t]
constraints += [x_cvxpy >= 0]

# formulate problem 
prob_cvxpy = cp.Problem(objective, constraints)
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -29.61311182548822 
Cvxpy x:  [6.82227779e+00 3.75795768e-10 3.95958213e-10 2.97132557e-10
 4.45313658e-10]


We solve the same problem for the MRO uncertainty set with $K= 1$ and power = 1. We note that this gives a different reformulation, and upper bounds the solution for the normal Ellipsoidal case. 

In [4]:
# formulate uncertainty set
n = 5
u = lropt.UncertainParameter(n,
                             uncertainty_set=lropt.MRO(K = 1, data = data,p=2,
                                                       rho=1, train = False))
# formulate cvxpy variables
x_m = cp.Variable(n)
t = cp.Variable()

# formulate objective
objective = cp.Minimize(t)

# formulate constraints 
constraints = [cp.maximum(a@x_m - d@x_m, a@x_m - d@(3*u+3)) <= t]
constraints += [x_m >= 0]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("MRO objective value: ", prob_robust.objective.value, "\nMRO x: ", x_m.value)

MRO objective value:  -11.03324770206946 
MRO x:  [1.25546009e+01 4.45637802e-11 5.20161882e-11 3.29681211e-11
 6.38461760e-11]


We assert the equivalence to the direct reformulation of the MRO problem. 

In [5]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
t = cp.Variable()

# formulate objective
objective = cp.Minimize(t)

# formulate constraints
constraints = [a@x_cvxpy -d@x_cvxpy + cp.norm(3*d,2) <= t]
constraints += [a@x_cvxpy -3*d@np.ones(n) + np.mean(data, axis = 0)@(-3*d) + cp.norm(3*d,2) <= t]
constraints += [x_cvxpy >= 0]

# formulate problem 
prob_cvxpy = cp.Problem(objective, constraints)
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_m.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -11.033247698109944 
Cvxpy x:  [1.25546009e+01 8.27847514e-10 8.35377661e-10 6.61619693e-10
 8.63321297e-10]
